In [1]:
!git clone https://github.com/zhangbo2008/xuni8
!git clone  https://huggingface.co/camenduru/Wav2Lip

Cloning into 'xuni8'...
remote: Enumerating objects: 2380, done.
remote: Counting objects: 100% (891/891), done.
remote: Compressing objects: 100% (886/886), done.
remote: Total 2380 (delta 5), reused 887 (delta 4), pack-reused 1489
Receiving objects: 100% (2380/2380), 183.10 MiB | 33.80 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Updating files: 100% (2859/2859), done.
Cloning into 'Wav2Lip'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (72/72), 442.42 KiB | 6.51 MiB/s, done.
Filtering content: 100% (6/6), 936.88 MiB | 70.47 MiB/s, done.


In [2]:
%cd /kaggle/working/xuni8

/kaggle/working/xuni8


In [7]:
# 先需要训练一个syncnet给4.py用. 目标训练一个音频视频是否同步的分类器. 输出是不是同步的概率.
# import trl

from os.path import dirname, join, basename, isfile
from tqdm import tqdm

from models import SyncNet_color as SyncNet
import audio
print(1)
import torch
from torch import nn
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils import data as data_utils
import numpy as np
print(2)
from glob import glob

import os, random, cv2, argparse
from hparams import hparams, get_image_list

parser = argparse.ArgumentParser(description='Code to train the expert lip-sync discriminator')

parser.add_argument("--data_root", help="Root folder of the preprocessed LRS2 dataset", required=False)

parser.add_argument('--checkpoint_dir', help='Save checkpoints to this directory', required=False, type=str)
parser.add_argument('--checkpoint_path', help='Resumed from this checkpoint', default=None, type=str)

args = parser.parse_args('') # kaggle需要传入一个空字符串.

ppp='/kaggle/working/Wav2Lip/'
args.data_root='lrs2_preprocessed/LRS2_partly'
args.checkpoint_dir='./tmp2'
args.checkpoint_path=ppp+'checkpoints/lipsync_expert.pth'

global_step = 10
global_epoch = 10
use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))

syncnet_T = 5
syncnet_mel_step_size = 16
hparams.syncnet_checkpoint_interval=300
hparams.num_workers=0
hparams.syncnet_batch_size=1000
hparams.syncnet_lr=3e-5
class Dataset(object):
    def __init__(self, split):
        # self.all_videos = get_image_list(args.data_root, split)
        self.all_videos =glob('my_data_preprocessed/20171116/*')
        print(self.all_videos)
    def get_frame_id(self, frame):
        return int(basename(frame).split('.')[0])

    def get_window(self, start_frame):
        start_id = self.get_frame_id(start_frame)
        vidname = dirname(start_frame)

        window_fnames = []
        for frame_id in range(start_id, start_id + syncnet_T):
            frame = join(vidname, '{}.jpg'.format(frame_id))
            if not isfile(frame):
                return None
            window_fnames.append(frame)
        return window_fnames

    def crop_audio_window(self, spec, start_frame):
        # num_frames = (T x hop_size * fps) / sample_rate
        start_frame_num = self.get_frame_id(start_frame)
        start_idx = int(80. * (start_frame_num / float(hparams.fps)))

        end_idx = start_idx + syncnet_mel_step_size

        return spec[start_idx : end_idx, :]


    def __len__(self):
        return len(self.all_videos)

    def __getitem__(self, idx):
        while 1:
            idx = random.randint(0, len(self.all_videos) - 1)
            vidname = self.all_videos[idx] # 随便抽取一个视频.

            img_names = list(glob(join(vidname, '*.jpg')))
            if len(img_names) <= 3 * syncnet_T:
                continue
            img_name = random.choice(img_names)
            wrong_img_name = random.choice(img_names)
            while wrong_img_name == img_name:
                wrong_img_name = random.choice(img_names)
            #选一个真或者假照片.
            if random.choice([True, False]):
                y = torch.ones(1).float()
                chosen = img_name
            else:
                y = torch.zeros(1).float()
                chosen = wrong_img_name

            window_fnames = self.get_window(chosen)
            if window_fnames is None:
                continue

            window = []
            all_read = True
            for fname in window_fnames:
                img = cv2.imread(fname)
                if img is None:
                    all_read = False
                    break
                try:
                    img = cv2.resize(img, (hparams.img_size, hparams.img_size))
                except Exception as e:
                    all_read = False
                    break

                window.append(img)

            if not all_read: continue

            if 1:
#                 print(88888888888888888,vidname)
                aaa=list(glob(join(vidname, '*.wav')))[0]
                wavpath = join(vidname, "audio.wav")
#                 print(aaa,333333333)
                wav = audio.load_wav(aaa, hparams.sample_rate) # 输入音频原始的sr不用管, 这里面设置好我们需要的sr即可.16000.
#                 print(222222222)
                orig_mel = audio.melspectrogram(wav).T
#             except Exception as e:
#                 continue
#=======根据片段拿到真实的读音.
            mel = self.crop_audio_window(orig_mel.copy(), img_name)

            if (mel.shape[0] != syncnet_mel_step_size):
                continue

            # H x W x 3 * T
            x = np.concatenate(window, axis=2) / 255.
            x = x.transpose(2, 0, 1)
            x = x[:, x.shape[1]//2:] #################????????????????????????????????????????????????????为啥要切一半呢?????????????????我理解是人脸嘴的部分一定在图片的下半部分, 所以去掉上面, 会加速网络收敛.

            x = torch.FloatTensor(x)
            mel = torch.FloatTensor(mel.T).unsqueeze(0)

            return x, mel, y

logloss = nn.BCELoss()
def cosine_loss(a, v, y):
    d = nn.functional.cosine_similarity(a, v)
    loss = logloss(d.unsqueeze(1), y)

    return loss

def train(device, model, train_data_loader, test_data_loader, optimizer,
          checkpoint_dir=None, checkpoint_interval=None, nepochs=None):

    global global_step, global_epoch
    resumed_step = global_step
    
    while global_epoch < nepochs:
        running_loss = 0.
        prog_bar = tqdm(enumerate(train_data_loader))
#         print(global_epoch,'global_epoch')
        for step, (x, mel, y) in prog_bar:
#             print(step,'step')
            model.train()
            optimizer.zero_grad()

            # Transform data to CUDA device
            x = x.to(device)

            mel = mel.to(device)

            a, v = model(mel, x) # a:audio v:video 都是512向量.
            y = y.to(device)

            loss = cosine_loss(a, v, y)
            loss.backward()
            optimizer.step()

            global_step += 1
            cur_session_steps = global_step - resumed_step
            running_loss += loss.item()

            if global_step == 1 or global_step % checkpoint_interval == 0:
                save_checkpoint(
                    model, optimizer, global_step, checkpoint_dir, global_epoch)

            if global_step % hparams.syncnet_eval_interval == 0:
                with torch.no_grad():
                    eval_model(test_data_loader, global_step, device, model, checkpoint_dir)

            prog_bar.set_description(f'Loss: {running_loss / (step + 1)},epoch:{global_step}')

        global_epoch += 1

def eval_model(test_data_loader, global_step, device, model, checkpoint_dir):
    eval_steps = 1400
    print('Evaluating for {} steps'.format(eval_steps))
    losses = []
    while 1:
        for step, (x, mel, y) in enumerate(test_data_loader):

            model.eval()

            # Transform data to CUDA device
            x = x.to(device)

            mel = mel.to(device)

            a, v = model(mel, x)
            y = y.to(device)

            loss = cosine_loss(a, v, y)
            losses.append(loss.item())

            if step > eval_steps: break

        averaged_loss = sum(losses) / len(losses)
        print(averaged_loss)

        return

def save_checkpoint(model, optimizer, step, checkpoint_dir, epoch):

    checkpoint_path = join(
        checkpoint_dir, "checkpoint_step{:09d}.pth".format(global_step))
    optimizer_state = optimizer.state_dict() if hparams.save_optimizer_state else None
    torch.save({
        "state_dict": model.state_dict(),
        "optimizer": optimizer_state,
        "global_step": step,
        "global_epoch": epoch,
    }, checkpoint_path)
    print("Saved checkpoint:", checkpoint_path)

def _load(checkpoint_path):
    if use_cuda:
        checkpoint = torch.load(checkpoint_path)
    else:
        checkpoint = torch.load(checkpoint_path,
                                map_location=lambda storage, loc: storage)
    return checkpoint

def load_checkpoint(path, model, optimizer, reset_optimizer=False):
    global global_step
    global global_epoch

    print("Load checkpoint from: {}".format(path))
    checkpoint = _load(path)
    model.load_state_dict(checkpoint["state_dict"])
    if not reset_optimizer:
        optimizer_state = checkpoint["optimizer"]
        if optimizer_state is not None:
            print("Load optimizer state from {}".format(path))
            optimizer.load_state_dict(checkpoint["optimizer"])
    global_step = checkpoint["global_step"]
    global_epoch = checkpoint["global_epoch"]

    return model

if __name__ == "__main__":
    checkpoint_dir = args.checkpoint_dir
    checkpoint_path = args.checkpoint_path

    if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)

    # Dataset and Dataloader setup
    train_dataset = Dataset('train')
    test_dataset = Dataset('val')

    train_data_loader = data_utils.DataLoader(
        train_dataset, batch_size=hparams.syncnet_batch_size, shuffle=True,
        num_workers=hparams.num_workers)

    test_data_loader = data_utils.DataLoader(
        test_dataset, batch_size=hparams.syncnet_batch_size,
        num_workers=8)

    device = torch.device("cuda" if use_cuda else "cpu")

    # Model
    model = SyncNet().to(device)
    print('total trainable params {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

    optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad],
                           lr=hparams.syncnet_lr)

    if checkpoint_path is not None:
        load_checkpoint(checkpoint_path, model, optimizer, reset_optimizer=False)

    train(device, model, train_data_loader, test_data_loader, optimizer,
          checkpoint_dir=checkpoint_dir,
          checkpoint_interval=hparams.syncnet_checkpoint_interval,
          nepochs=hparams.nepochs)


1
2
use_cuda: True
['my_data_preprocessed/20171116/section_2_028', 'my_data_preprocessed/20171116/section_3_000', 'my_data_preprocessed/20171116/section_1_081', 'my_data_preprocessed/20171116/section_2_033', 'my_data_preprocessed/20171116/section_1_049', 'my_data_preprocessed/20171116/section_2_024', 'my_data_preprocessed/20171116/section_2_020', 'my_data_preprocessed/20171116/section_1_024', 'my_data_preprocessed/20171116/section_1_068', 'my_data_preprocessed/20171116/section_2_016']
['my_data_preprocessed/20171116/section_2_028', 'my_data_preprocessed/20171116/section_3_000', 'my_data_preprocessed/20171116/section_1_081', 'my_data_preprocessed/20171116/section_2_033', 'my_data_preprocessed/20171116/section_1_049', 'my_data_preprocessed/20171116/section_2_024', 'my_data_preprocessed/20171116/section_2_020', 'my_data_preprocessed/20171116/section_1_024', 'my_data_preprocessed/20171116/section_1_068', 'my_data_preprocessed/20171116/section_2_016']
total trainable params 16435072
Load ch

Loss: 0.767553985118866,epoch:1751231: : 1it [00:00,  2.95it/s]
Loss: 0.877458393573761,epoch:1751232: : 1it [00:00,  3.18it/s]
Loss: 0.4243386685848236,epoch:1751233: : 1it [00:00,  2.62it/s]
Loss: 0.5802802443504333,epoch:1751234: : 1it [00:00,  2.75it/s]
Loss: 0.7357434630393982,epoch:1751235: : 1it [00:00,  2.46it/s]
Loss: 0.42107173800468445,epoch:1751236: : 1it [00:00,  2.40it/s]
Loss: 0.3131580948829651,epoch:1751237: : 1it [00:00,  2.59it/s]
Loss: 0.3601432740688324,epoch:1751238: : 1it [00:00,  2.84it/s]
Loss: 0.5840126872062683,epoch:1751239: : 1it [00:00,  3.62it/s]
Loss: 0.42108169198036194,epoch:1751240: : 1it [00:00,  2.39it/s]
Loss: 0.5986115336418152,epoch:1751241: : 1it [00:00,  2.24it/s]
Loss: 0.8161717653274536,epoch:1751242: : 1it [00:00,  2.65it/s]
Loss: 0.2917346954345703,epoch:1751243: : 1it [00:00,  2.79it/s]
Loss: 0.43913522362709045,epoch:1751244: : 1it [00:00,  1.91it/s]
Loss: 0.37753230333328247,epoch:1751245: : 1it [00:00,  2.47it/s]
Loss: 0.582250535488128

Saved checkpoint: ./tmp2/checkpoint_step001751400.pth


Loss: 0.34751930832862854,epoch:1751401: : 1it [00:00,  2.27it/s]
Loss: 0.5574396848678589,epoch:1751402: : 1it [00:00,  1.76it/s]
Loss: 0.19069352746009827,epoch:1751403: : 1it [00:00,  2.57it/s]
Loss: 10.310107231140137,epoch:1751404: : 1it [00:00,  1.99it/s]
Loss: 0.48897430300712585,epoch:1751405: : 1it [00:00,  1.74it/s]
Loss: 0.3920682966709137,epoch:1751406: : 1it [00:00,  3.63it/s]
Loss: 0.29813045263290405,epoch:1751407: : 1it [00:00,  2.74it/s]
Loss: 0.6438532471656799,epoch:1751408: : 1it [00:00,  2.22it/s]
Loss: 0.4431682527065277,epoch:1751409: : 1it [00:00,  2.54it/s]
Loss: 0.1764533817768097,epoch:1751410: : 1it [00:00,  1.77it/s]
Loss: 0.5738363862037659,epoch:1751411: : 1it [00:00,  2.52it/s]
Loss: 0.14858652651309967,epoch:1751412: : 1it [00:00,  2.19it/s]
Loss: 0.36434322595596313,epoch:1751413: : 1it [00:00,  2.15it/s]
Loss: 0.6783915758132935,epoch:1751414: : 1it [00:00,  1.61it/s]
Loss: 0.628143310546875,epoch:1751415: : 1it [00:00,  2.66it/s]
Loss: 0.447629839181

KeyboardInterrupt: 

In [10]:
# 学习训练代码.
import glob
# print(glob.glob('./lrs2_preprocessed/LRS2_partly/*/*.jpg'))
# print(glob.glob('./*.*'))

from os.path import dirname, join, basename, isfile
from tqdm import tqdm

from models import SyncNet_color as SyncNet
from models import Wav2Lip as Wav2Lip
import audio

import torch
from torch import nn
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils import data as data_utils
import numpy as np

from glob import glob

import os, random, cv2, argparse
from hparams import hparams, get_image_list

parser = argparse.ArgumentParser(description='Code to train the Wav2Lip model without the visual quality discriminator')

parser.add_argument("--data_root", help="Root folder of the preprocessed LRS2 dataset", required=False, type=str)

parser.add_argument('--checkpoint_dir', help='Save checkpoints to this directory', required=False, type=str)
parser.add_argument('--syncnet_checkpoint_path', help='Load the pre-trained Expert discriminator', required=False, type=str)

parser.add_argument('--checkpoint_path', help='Resume from this checkpoint', default=None, type=str)
ppp='/kaggle/working/Wav2Lip/'
args = parser.parse_args('')
# args.data_root='lrs2_preprocessed'
args.checkpoint_dir='newmodel'
args.syncnet_checkpoint_path=ppp+'checkpoints/lipsync_expert.pth'
args.checkpoint_path=ppp+'checkpoints/wav2lip_gan.pth'

hparams.checkpoint_interval=300

hparams.num_workers=0
hparams.syncnet_batch_size=1000
hparams.syncnet_lr=3e-5









global_step = 0
global_epoch = 0
use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))

syncnet_T = 5               # 视频片段长度
syncnet_mel_step_size = 16  # 音频片段长度

class Dataset(object):
    def __init__(self, split):
        # self.all_videos = get_image_list(args.data_root, split)
        self.all_videos =glob('my_data_preprocessed/20171116/*')
        print(self.all_videos)
    def get_frame_id(self, frame):
        return int(basename(frame).split('.')[0])

    def get_window(self, start_frame):
        start_id = self.get_frame_id(start_frame)
        vidname = dirname(start_frame)

        window_fnames = []
        for frame_id in range(start_id, start_id + syncnet_T):
            frame = join(vidname, '{}.jpg'.format(frame_id)) # 取5帧
            if not isfile(frame):
                return None
            window_fnames.append(frame)
        return window_fnames

    def read_window(self, window_fnames):
        if window_fnames is None: return None
        window = []
        for fname in window_fnames:
            img = cv2.imread(fname)
            if img is None:
                return None
            try:
                img = cv2.resize(img, (hparams.img_size, hparams.img_size))
            except Exception as e:
                return None

            window.append(img)

        return window

    def crop_audio_window(self, spec, start_frame):
        if type(start_frame) == int:
            start_frame_num = start_frame
        else:
            start_frame_num = self.get_frame_id(start_frame) # 0-indexing ---> 1-indexing
        start_idx = int(80. * (start_frame_num / float(hparams.fps)))
        # mel普一秒是80个数, (start_frame_num / float(hparams.fps) 当前流逝时间.
        end_idx = start_idx + syncnet_mel_step_size

        return spec[start_idx : end_idx, :]

    def get_segmented_mels(self, spec, start_frame):
        mels = []
        assert syncnet_T == 5
        start_frame_num = self.get_frame_id(start_frame) + 1 # 0-indexing ---> 1-indexing
        if start_frame_num - 2 < 0: return None
        for i in range(start_frame_num, start_frame_num + syncnet_T):
            m = self.crop_audio_window(spec, i - 2)
            if m.shape[0] != syncnet_mel_step_size:
                return None
            mels.append(m.T)

        mels = np.asarray(mels)

        return mels

    def prepare_window(self, window):
        # 3 x T x H x W
        x = np.asarray(window) / 255.
        x = np.transpose(x, (3, 0, 1, 2))

        return x

    def __len__(self):
        return len(self.all_videos)

    def __getitem__(self, idx):
        while 1:
            idx = random.randint(0, len(self.all_videos) - 1)
            vidname = self.all_videos[idx]
            img_names = list(glob(join(vidname, '*.jpg'))) # 获取视频的所有贞.
            if len(img_names) <= 3 * syncnet_T:
                continue
            
            img_name = random.choice(img_names)
            wrong_img_name = random.choice(img_names)
            while wrong_img_name == img_name:
                wrong_img_name = random.choice(img_names)
#------根据抽样得到窗口贞.
            window_fnames = self.get_window(img_name)
            wrong_window_fnames = self.get_window(wrong_img_name)
            if window_fnames is None or wrong_window_fnames is None:
                continue

            window = self.read_window(window_fnames)
            if window is None:
                continue

            wrong_window = self.read_window(wrong_window_fnames)
            if wrong_window is None:
                continue

            try:
                aaa=list(glob(join(vidname, '*.wav')))[0]
                wavpath = join(vidname, "audio.wav")
                wav = audio.load_wav(aaa, hparams.sample_rate)

                orig_mel = audio.melspectrogram(wav).T
            except Exception as e:
                continue
            # mel 当前时间点, 0.2秒钟的音频特征.
            mel = self.crop_audio_window(orig_mel.copy(), img_name)
            
            if (mel.shape[0] != syncnet_mel_step_size):
                continue
            # indiv_mels是上一个mel的后续5个.
            indiv_mels = self.get_segmented_mels(orig_mel.copy(), img_name)
            if indiv_mels is None: continue

            window = self.prepare_window(window)
            y = window.copy() #======作为标签.
            window[:, :, window.shape[2]//2:] = 0. # 遮挡住视频的下半部分人脸.
#   window: 34帧到39帧, mel:34的音频. inv_mel: 35到39的音频. wrong_window: 58到62的帧.
            wrong_window = self.prepare_window(wrong_window)
            x = np.concatenate([window, wrong_window], axis=0)

            x = torch.FloatTensor(x)
            mel = torch.FloatTensor(mel.T).unsqueeze(0)
            indiv_mels = torch.FloatTensor(indiv_mels).unsqueeze(1)
            y = torch.FloatTensor(y)
            return x, indiv_mels, mel, y

def save_sample_images(x, g, gt, global_step, checkpoint_dir):
    x = (x.detach().cpu().numpy().transpose(0, 2, 3, 4, 1) * 255.).astype(np.uint8)
    g = (g.detach().cpu().numpy().transpose(0, 2, 3, 4, 1) * 255.).astype(np.uint8)
    gt = (gt.detach().cpu().numpy().transpose(0, 2, 3, 4, 1) * 255.).astype(np.uint8)

    refs, inps = x[..., 3:], x[..., :3]
    folder = join(checkpoint_dir, "samples_step{:09d}".format(global_step))
    if not os.path.exists(folder): os.mkdir(folder)
    collage = np.concatenate((refs, inps, g, gt), axis=-2)
    for batch_idx, c in enumerate(collage):
        for t in range(len(c)):
            cv2.imwrite('{}/{}_{}.jpg'.format(folder, batch_idx, t), c[t])

logloss = nn.BCELoss()
def cosine_loss(a, v, y):
    d = nn.functional.cosine_similarity(a, v)
    loss = logloss(d.unsqueeze(1), y)

    return loss

device = torch.device("cuda" if use_cuda else "cpu")
syncnet = SyncNet().to(device)
for p in syncnet.parameters():
    p.requires_grad = False

recon_loss = nn.L1Loss()
def get_sync_loss(mel, g):
    g = g[:, :, :, g.size(3)//2:]
    g = torch.cat([g[:, :, i] for i in range(syncnet_T)], dim=1)
    # B, 3 * T, H//2, W
    a, v = syncnet(mel, g)
    y = torch.ones(g.size(0), 1).float().to(device)
    return cosine_loss(a, v, y)

def train(device, model, train_data_loader, test_data_loader, optimizer,
          checkpoint_dir=None, checkpoint_interval=None, nepochs=None):

    global global_step, global_epoch
    resumed_step = global_step
 
    while global_epoch < nepochs:
        print('Starting Epoch: {}'.format(global_epoch))
        running_sync_loss, running_l1_loss = 0., 0.5
        prog_bar = tqdm(enumerate(train_data_loader))
        for step, (x, indiv_mels, mel, gt) in prog_bar:
            model.train()
            optimizer.zero_grad()

            # Move data to CUDA device
            x = x.to(device)
            mel = mel.to(device)
            indiv_mels = indiv_mels.to(device)
            gt = gt.to(device)
# 输入音频和x. 输出g
            g = model(indiv_mels, x) # 输入后续mel和当前图片, 计算后续的图片.

            if hparams.syncnet_wt > 0.:
                sync_loss = get_sync_loss(mel, g) # 这个是同步分数, 用来判定音频和视频是否同步.
            else:
                sync_loss = 0.

            l1loss = recon_loss(g, gt) # 重构loss.

            loss = hparams.syncnet_wt * sync_loss + (1 - hparams.syncnet_wt) * l1loss
            loss.backward()
            optimizer.step()
            if 0:#========禁掉eval
             if global_step % checkpoint_interval == 0:
                save_sample_images(x, g, gt, global_step, checkpoint_dir)

            global_step += 1
            cur_session_steps = global_step - resumed_step

            running_l1_loss += l1loss.item()
            if hparams.syncnet_wt > 0.:
                running_sync_loss += sync_loss.item()
            else:
                running_sync_loss += 0.

            if global_step == 1 or global_step % checkpoint_interval == 0:
                save_checkpoint(
                    model, optimizer, global_step, checkpoint_dir, global_epoch)
            if 0:#==========禁掉eval
                if global_step == 1 or global_step % hparams.eval_interval == 0:
                    with torch.no_grad():
                        average_sync_loss = eval_model(test_data_loader, global_step, device, model, checkpoint_dir)

                        if average_sync_loss < .75:
                            hparams.set_hparam('syncnet_wt', 0.01) # without image GAN a lesser weight is sufficient
            # print(f'loss:{loss},global_step:{global_step}')
            # prog_bar.set_description('L1: {}, Sync Loss: {}'.format(running_l1_loss / (step + 1),
            #                                                         running_sync_loss / (step + 1)))
            prog_bar.set_description(f'loss:{loss},global_step:{global_step}')

        global_epoch += 1
        

def eval_model(test_data_loader, global_step, device, model, checkpoint_dir):
    eval_steps = 700
    print('Evaluating for {} steps'.format(eval_steps))
    sync_losses, recon_losses = [], []
    step = 0
    while 1:
        for x, indiv_mels, mel, gt in test_data_loader:
            step += 1
            model.eval()

            # Move data to CUDA device
            x = x.to(device)
            gt = gt.to(device)
            indiv_mels = indiv_mels.to(device)
            mel = mel.to(device)

            g = model(indiv_mels, x)

            sync_loss = get_sync_loss(mel, g)
            l1loss = recon_loss(g, gt)

            sync_losses.append(sync_loss.item())
            recon_losses.append(l1loss.item())

            if step > eval_steps: 
                averaged_sync_loss = sum(sync_losses) / len(sync_losses)
                averaged_recon_loss = sum(recon_losses) / len(recon_losses)

                print('L1: {}, Sync loss: {}'.format(averaged_recon_loss, averaged_sync_loss))

                return averaged_sync_loss

def save_checkpoint(model, optimizer, step, checkpoint_dir, epoch):

    checkpoint_path = join(
        checkpoint_dir, "checkpoint_step{:09d}.pth".format(global_step))
    optimizer_state = optimizer.state_dict() if hparams.save_optimizer_state else None
    torch.save({
        "state_dict": model.state_dict(),
        "optimizer": optimizer_state,
        "global_step": step,
        "global_epoch": epoch,
    }, checkpoint_path)
    print("Saved checkpoint:", checkpoint_path)


def _load(checkpoint_path):
    if use_cuda:
        checkpoint = torch.load(checkpoint_path)
    else:
        checkpoint = torch.load(checkpoint_path,
                                map_location=lambda storage, loc: storage)
    return checkpoint

def load_checkpoint(path, model, optimizer, reset_optimizer=False, overwrite_global_states=True):
    global global_step
    global global_epoch

    print("Load checkpoint from: {}".format(path))
    checkpoint = _load(path)
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)
    if not reset_optimizer:
        optimizer_state = checkpoint["optimizer"]
        if optimizer_state is not None:
            print("Load optimizer state from {}".format(path))
            optimizer.load_state_dict(checkpoint["optimizer"])
    if overwrite_global_states:
        global_step = checkpoint["global_step"]
        global_epoch = checkpoint["global_epoch"]

    return model

if __name__ == "__main__":
    checkpoint_dir = args.checkpoint_dir

    # Dataset and Dataloader setup
    train_dataset = Dataset('train')
    test_dataset = Dataset('val')
    hparams.batch_size=1
    hparams.num_workers=0
    train_data_loader = data_utils.DataLoader(
        train_dataset, batch_size=hparams.batch_size, shuffle=True,
        num_workers=hparams.num_workers)

    test_data_loader = data_utils.DataLoader(
        test_dataset, batch_size=hparams.batch_size,
        num_workers=hparams.num_workers)

    device = torch.device("cuda" if use_cuda else "cpu")

    # Model
    model = Wav2Lip().to(device)
    print('total trainable params {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

    optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad],
                           lr=hparams.initial_learning_rate)

    if args.checkpoint_path is not None: # 加载wav2lip 的权重
        load_checkpoint(args.checkpoint_path, model, optimizer, reset_optimizer=False)
        
    load_checkpoint(args.syncnet_checkpoint_path, syncnet, None, reset_optimizer=True, overwrite_global_states=False)  # 加载sync打分模型.
#===========加载完syncnet和model
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)

    # Train!
    train(device, model, train_data_loader, test_data_loader, optimizer,
              checkpoint_dir=checkpoint_dir,
              checkpoint_interval=hparams.checkpoint_interval,
              nepochs=hparams.nepochs)

use_cuda: True
['my_data_preprocessed/20171116/section_2_028', 'my_data_preprocessed/20171116/section_3_000', 'my_data_preprocessed/20171116/section_1_081', 'my_data_preprocessed/20171116/section_2_033', 'my_data_preprocessed/20171116/section_1_049', 'my_data_preprocessed/20171116/section_2_024', 'my_data_preprocessed/20171116/section_2_020', 'my_data_preprocessed/20171116/section_1_024', 'my_data_preprocessed/20171116/section_1_068', 'my_data_preprocessed/20171116/section_2_016']
['my_data_preprocessed/20171116/section_2_028', 'my_data_preprocessed/20171116/section_3_000', 'my_data_preprocessed/20171116/section_1_081', 'my_data_preprocessed/20171116/section_2_033', 'my_data_preprocessed/20171116/section_1_049', 'my_data_preprocessed/20171116/section_2_024', 'my_data_preprocessed/20171116/section_2_020', 'my_data_preprocessed/20171116/section_1_024', 'my_data_preprocessed/20171116/section_1_068', 'my_data_preprocessed/20171116/section_2_016']
total trainable params 36298035
Load checkp

loss:0.09145378321409225,global_step:258010: : 10it [00:01,  7.49it/s]


Starting Epoch: 89


loss:0.08842376619577408,global_step:258020: : 10it [00:01,  7.57it/s]


Starting Epoch: 90


...

Starting Epoch: 102


loss:0.04783051460981369,global_step:258150: : 10it [00:01,  8.44it/s]


Starting Epoch: 103


loss:0.04739758372306824,global_step:258158: : 8it [00:01,  7.90it/s] 


KeyboardInterrupt: 